Importer les packages

In [159]:
# importer les modules python nécessaires
from itertools import product
import os
from collections import defaultdict
from statistics import mean, stdev
import re

# Pour lire le fichier fasta
from Bio import SeqIO
import subprocess

# importer numpy
import numpy as np

# importer les modules scikit-learn nécessaires
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score

1. Repliement avec RNAfold

In [160]:

def executer_rnafold(fichier_fasta, fichier_output):
    # opening the fasta file

    with open(fichier_output, "w") as output_file:
        for seq_record in SeqIO.parse(fichier_fasta, "fasta"):
            # extracting the sequence (.seq)
            sequence = str(seq_record.seq)
            # 1. Lancer RNAfold et récupérer son output stdout dans la variable string repliement_fasta.
            process = subprocess.Popen(['RNAfold'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            [stdout, stderr] = process.communicate(input=sequence)
            # Write the sequence name (.id)
            output_file.write(f">{seq_record.id}\n")
            # 2. Enregistrer repliement_fasta dans le fichier fichier_output
            output_file.write(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")
        
        # print(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")

In [161]:
# Our given fasta file in a list 
fasta_files = ["Fasta/hsa_hairpin.fasta", "Fasta/hsa_hairpin_neg1.fasta", "Fasta/hsa_hairpin_neg2.fasta"]
# looping through the files to get the folds files
for file in fasta_files :
    # make the name of the output file from its input file
    file_output = f"Fasta/repliments/{file.split('/')[1].split('.')[0]}_fold.fasta"
    # check if the output file exist 
    if not os.path.isfile(file_output):
        executer_rnafold(file, file_output)

In [162]:
def traiter_fasta_fold(fichier_fold):
    
    repliments = defaultdict(list)

    # 1. Lire le fichier fichier_fold et récupérer son contenu dans une liste lignes
    with open(fichier_fold, 'r') as file:
        lignes = file.readlines()

    # supprimer "\n" à la fin des lignes
    lignes_n = [ligne.strip() for ligne in lignes]

    # 2. Traiter repliement_fasta pour créer un dictionnaire de listes defaultdict(list)
    i = 0
    while i < len(lignes_n):
        # si la ligne commence par '>' donc on est au debut du nom de la sequence
        if lignes_n[i].startswith('>'):
            nom_sequence = lignes_n[i][1:]
            # mettre en lowercase
            sequence = lignes_n[i+1].lower()
            structure = lignes_n[i+2]
            # enregistrer les sequences et structures dans le dictionnaire
            repliments[nom_sequence] = [sequence, structure]
            # saut de 3 lignes : nom + sequence + repliement
            i += 3
        else:
        # sinon saut d'une ligne jusqu'a trouver le '>' du debut
            i += 1

    return repliments
   


In [163]:
# adresse fichier des repliments 
output_path = "Fasta/repliments/"
# pour tester la fonction traiter_fasta_fold() 
# j'ai opter pour un dictionnaire qui contient les noms des fichiers output comme cle et le dictionnaire generer par la fonction comme valeur

repliments = defaultdict()

for file in os.listdir(output_path):
    # On extrait les cles du nom de leur fichier respective
    name = f"dict{file.split('hairpin')[1].split('.fasta')[0]}"
    # print(f"dict{file.split('hairpin')[1].split('.fasta')[0]}")
    repliments[name] = traiter_fasta_fold(f"{output_path}{file}")
    
print(repliments.keys())
repliments.items()

dict_keys(['dict_fold', 'dict_neg1_fold', 'dict_neg2_fold'])


dict_items([('dict_fold', defaultdict(<class 'list'>, {'hsa-mir-5683': ['ggagcuuguuacagaugcagauucucugacuucuuacugcaccagugaagucaggaucugcauuugaauaagaccc', '((..((((((.(((((((((((.(.((((((((..((((...))))))))))))))))))))))))))))))..))'], 'hsa-mir-1587': ['uuugggcugggcuggguugggcaguucuucugcuggacucaccugugaccagc', '.((((..((((.((((((.(((((.....))))).))))))))))...)))).'], 'hsa-mir-572': ['gucgaggccguggcccggaaguggucggggccgcugcgggcggaagggcgccugugcuucguccgcucggcgguggcccagccaggcccgcggga', '.......((((((.((.....((((..(((((((((((((((((.((((......))))..))))))).)))))))))).)))))).))))))..'], 'hsa-mir-2116': ['gaccuaggcuagggguucuuagcauaggaggucuucccaugcuaagaaguccucccaugccaagaacucccagacuagga', '..(((((.((.(((((((((.((((.(((((.((((.........)))).))))).)))).))))))))).)).))))).'], 'hsa-mir-3675': ['ggaugauaaguuauggggcuucuguagagauuucuaugagaacaucucuaaggaacucccccaaacugaauuc', '((((....((((.(((((.((((.(((((((((((...)))).))))))).))))...)))))))))..))))'], 'hsa-mir-6724-3': ['cgcugcgcuucugggcccgcggcgggcguggggcugcccggg

2. Calcul du nombre d’occurrences des triplets

In [164]:
# Ne pas modifier la fonction!

def get_all_triplets():    

    db = ["...", "..(", ".((", "(((", "((.", "(..", "(.(", ".(."]
    nd = ["a", "c", "g", "u"]

    return [i+j for i,j in product(nd, db)]

In [165]:
# Generer les sequences RNA et leur structures
triplets = get_all_triplets()
triplets

['a...',
 'a..(',
 'a.((',
 'a(((',
 'a((.',
 'a(..',
 'a(.(',
 'a.(.',
 'c...',
 'c..(',
 'c.((',
 'c(((',
 'c((.',
 'c(..',
 'c(.(',
 'c.(.',
 'g...',
 'g..(',
 'g.((',
 'g(((',
 'g((.',
 'g(..',
 'g(.(',
 'g.(.',
 'u...',
 'u..(',
 'u.((',
 'u(((',
 'u((.',
 'u(..',
 'u(.(',
 'u.(.']

In [166]:
# Ne pas modifier la fonction!


def get_seq_triplets(sequence, structure):
   
    seq_triplets = defaultdict(int)

    assert len(sequence) == len(structure)

    struct = re.sub("\)", "(", structure)

    seq_tab = list(sequence)
    stc_tab = list(struct)

    for ind in range(len(seq_tab)):
        if ind == 0: 
            before = "."
        else: 
            before = stc_tab[ind-1]

        midlle = stc_tab[ind]

        if ind == len(seq_tab)-1: 
            after = "."
        else:
            after = stc_tab[ind+1]

        triplet = seq_tab[ind] + before + midlle + after
        seq_triplets[triplet] += 1

    return seq_triplets

In [167]:
seq_triplets = defaultdict()
# Calculer les occurences des triplets dans les sequences et structures
for repliment, name in zip(repliments.values(), repliments.keys()) :
    # print(repliment)
    # print(name)
    for sequence, structure in repliment.values():
        seq_triplets[name] = get_seq_triplets(sequence, structure)
        print(f"Occurrences of triplets in sequence {name}: {seq_triplets}")
seq_triplets

Occurrences of triplets in sequence dict_fold: defaultdict(None, {'dict_fold': defaultdict(<class 'int'>, {'g.((': 1, 'g((.': 3, 'a(..': 2, 'g..(': 1, 'c.((': 5, 'u(((': 16, 'g(((': 12, 'u((.': 2, 'a(.(': 1, 'a(((': 15, 'c(((': 6, 'u(.(': 2, 'c.(.': 1, 'c((.': 2, 'u(..': 1, 'u..(': 1, 'a.((': 1, 'c(..': 1, 'a...': 1, 'c..(': 2})})
Occurrences of triplets in sequence dict_fold: defaultdict(None, {'dict_fold': defaultdict(<class 'int'>, {'u..(': 2, 'u.((': 3, 'u(((': 5, 'g(((': 8, 'g((.': 5, 'g(..': 1, 'c..(': 1, 'c(.(': 1, 'u((.': 2, 'g(.(': 2, 'g.((': 2, 'c(((': 7, 'a(((': 4, 'u(..': 2, 'u...': 2, 'c...': 1, 'c.((': 2, 'g...': 1, 'a..(': 1, 'c(..': 1})})
Occurrences of triplets in sequence dict_fold: defaultdict(None, {'dict_fold': defaultdict(<class 'int'>, {'g...': 5, 'u...': 2, 'c...': 3, 'a...': 4, 'g..(': 4, 'c.((': 3, 'c(((': 17, 'g(((': 23, 'u(((': 5, 'g((.': 3, 'c(.(': 2, 'c((.': 4, 'g(..': 3, 'u.((': 2, 'u((.': 2, 'c(..': 2, 'g.((': 5, 'a((.': 1, 'a(.(': 2, 'u..(': 1, 'g(.(': 

defaultdict(None,
            {'dict_fold': defaultdict(int,
                         {'u.(.': 1,
                          'g(.(': 8,
                          'a.((': 6,
                          'g(((': 13,
                          'c((.': 1,
                          'c(..': 1,
                          'g..(': 2,
                          'c(((': 11,
                          'u((.': 9,
                          'u(((': 3,
                          'a(((': 1,
                          'g.((': 4,
                          'a(..': 1,
                          'a...': 2,
                          'c...': 3,
                          'g(..': 2,
                          'a..(': 1,
                          'a.(.': 1}),
             'dict_neg1_fold': defaultdict(int,
                         {'u..(': 1,
                          'g.((': 1,
                          'c(((': 4,
                          'u(((': 1,
                          'g((.': 4,
                          'a(..': 3,

In [168]:
# Ne pas modifier la fonction!

def calculer_Xu_triplets(repliements):
   
    liste_triplets = get_all_triplets()

    nb_sequences = len(repliements)
    nb_triplets = len(liste_triplets)

    triplets = np.zeros((nb_sequences, nb_triplets))
    
    for i, nom in enumerate(repliements):
        seq_triplets = get_seq_triplets(repliements[nom][0], repliements[nom][1])
        
        for j, triplet in enumerate(liste_triplets):
            if triplet in seq_triplets:
                triplets[i, j] = seq_triplets[triplet]
    
    return triplets

In [169]:

Xu_triplets = defaultdict()
for repliment, name in zip( repliments.values(), repliments.keys() ):
    #print(name)
    Xu_triplets[name] = calculer_Xu_triplets(repliment)
    print(f"Matrice de {name} :")
    print(Xu_triplets[name].shape)
    print(Xu_triplets[name])
print(Xu_triplets.keys())

Matrice de dict_fold :
(163, 32)
[[1. 0. 1. ... 1. 2. 0.]
 [0. 1. 0. ... 2. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 ...
 [3. 0. 2. ... 0. 1. 0.]
 [1. 2. 0. ... 2. 0. 0.]
 [2. 1. 6. ... 0. 0. 1.]]
Matrice de dict_neg1_fold :
(1917, 32)
[[ 2.  1.  0. ...  0.  0.  0.]
 [ 9.  4.  2. ...  1.  0.  0.]
 [12.  2.  0. ...  3.  1.  0.]
 ...
 [10.  3.  0. ...  0.  0.  0.]
 [10.  2.  0. ...  2.  0.  0.]
 [ 8.  1.  0. ...  0.  0.  0.]]
Matrice de dict_neg2_fold :
(1917, 32)
[[13.  0.  0. ...  0.  1.  0.]
 [ 3.  0.  1. ...  2.  0.  0.]
 [ 8.  5.  1. ...  1.  0.  0.]
 ...
 [ 8.  4.  4. ...  2.  0.  0.]
 [ 5.  2.  0. ...  0.  0.  0.]
 [ 3.  1.  1. ...  1.  3.  0.]]
dict_keys(['dict_fold', 'dict_neg1_fold', 'dict_neg2_fold'])


3. Construction du jeu de données d'entrainement

In [170]:

def construire_dataset(data1, data2):
        
    # 1. Construire X

    X = np.concatenate((data1, data2), axis=0)
   
    # 2. Mettre à l'échelle les valeurs de X entre 0 et 1 avec la méthode min-max
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    X = (X - X_min) / (X_max - X_min)

    # 3. Construire y
    # Étiquettes des instances de data1 = 0
    # Étiquettes des instances de data2 = 1
    y = np.concatenate((np.zeros(data1.shape[0]), np.ones(data2.shape[0])))

    assert X.shape[0] == y.shape[0]

    return X, y


In [171]:
x, y = construire_dataset(Xu_triplets["dict_fold"], Xu_triplets["dict_neg1_fold"])
print("Matrice de x : \n",x)
print("Matrice de x : \n",y)

Matrice de x : 
 [[0.03125    0.         0.125      ... 0.16666667 0.28571429 0.        ]
 [0.         0.125      0.         ... 0.33333333 0.         0.        ]
 [0.125      0.         0.         ... 0.         0.         0.        ]
 ...
 [0.3125     0.375      0.         ... 0.         0.         0.        ]
 [0.3125     0.25       0.         ... 0.33333333 0.         0.        ]
 [0.25       0.125      0.         ... 0.         0.         0.        ]]
Matrice de x : 
 [0. 0. 0. ... 1. 1. 1.]


4. Évaluation et entrainement des modèles supervisés

In [172]:
# fonction evaluer_cv_entrainer() en utilsant cross_validate() 

def evaluer_cv_entrainer(X, y, cv=5, modele="lsvc", mesure='f1_weighted'):
    if modele == "lsvc":
        classifieur = SVC(kernel='linear')
    elif modele == "nsvc":
        classifieur = SVC(kernel='rbf')
    elif modele == "mnb":
        classifieur = MultinomialNB()
    elif modele == "gnb":
        classifieur = GaussianNB()
    elif modele == "dtc":
        classifieur = DecisionTreeClassifier()

    # scores = cross_val_score(classifieur, X, y, cv=cv, scoring=mesure)
    scores = cross_validate(classifieur, X, y, cv=cv, scoring=mesure)

    classifieur.fit(X, y)

    return scores, classifieur


In [185]:
# Évaluer et entraîner un classifieur avec validation croisée en utilisant la fonction cross_val_score
""" scores, classifieur = evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='f1_weighted')
print(f"using cross_val_score, lsvc: {scores}") """

# Évaluer et entraîner un classifieur avec validation croisée en utilisant la fonction cross_validate
scores, classifieur = evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='f1_weighted')
print(f"Scores using cross_validate : \n{scores['test_score']}")

print("Classifieur :\n",classifieur)

Scores using cross_validate : 
[0.94047091 0.93463247 0.92670051 0.93000761 0.95218656]
Classifieur :
 SVC(kernel='linear')


5. Fonction principale

In [186]:

def entrainer_par_triplets(
    fasta_seq_pos,
    fasta_fold_pos,
    fasta_seq_neg,
    fasta_fold_neg,
    cv,
    modele,
    mesure="f1_weighted"):
    

    ## 1. Replier les séquences du set positif
    # Replier les séquences si le fichier fasta_fold_pos n'existe pas
    if not os.path.isfile(fasta_fold_pos):
        executer_rnafold(fasta_seq_pos, fasta_fold_pos)
    
    ## 2. Replier les séquences du set négatif
    # Replier les séquences si le fichier fasta_fold_neg n'existe pas
    if not os.path.isfile(fasta_fold_neg):
        executer_rnafold(fasta_seq_neg, fasta_fold_neg)


    ## 3. Traiter les repliements du set positif 
    repliements_pos = traiter_fasta_fold(fasta_fold_pos)

    ## 4. Traiter les repliements du set négatif
    repliements_neg1 = traiter_fasta_fold(fasta_fold_neg)

    ## 5. Calculer les triplets du set positif
    triplets_pos = calculer_Xu_triplets(repliements_pos)

    ## 6. Calculer les triplets du set négatif
    triplets_neg1 = calculer_Xu_triplets(repliements_neg1)

    ## 7. Construire le jeu d'entrainement X et y
    x, y = construire_dataset(triplets_pos, triplets_neg1)

    ## 8. Évaluer et entrainer le classifieur avec validation croisée
    scores, classifieur = evaluer_cv_entrainer(x, y, cv, modele, mesure)


    return scores, classifieur

In [187]:
# Cette cellule est dédiée pour tester entrainer_par_triplets()
fasta_seq_pos = "Fasta/hsa_hairpin.fasta"
fasta_fold_pos = "Fasta/repliments/hsa_hairpin_fold.fasta"
fasta_seq_neg = "Fasta/hsa_hairpin_neg1.fasta"
fasta_fold_neg = "Fasta/repliments/hsa_hairpin_neg1_fold.fasta"

scores, classifieur = entrainer_par_triplets(fasta_seq_pos, fasta_fold_pos, fasta_seq_neg, fasta_fold_neg, cv=10, modele="lsvc", mesure="f1_weighted")

scores['test_score']

array([0.93402367, 0.9382048 , 0.94252028, 0.92587413, 0.94160662,
       0.92587413, 0.91714172, 0.94160662, 0.94160662, 0.96193586])

6. Comparaison des modèles

In [176]:

def comparer_modeles(
    fasta_seq_pos,
    fasta_fold_pos,
    fasta_seq_neg,
    fasta_fold_neg,
    cv,
    modeles=["lsvc", "nsvc", "mnb", "gnb", "dtc"],
    mesure="f1_weighted"):

    moyennes = []
    ecarts_types = []

    for modele in modeles:
        scores, _ = entrainer_par_triplets(fasta_seq_pos, fasta_fold_pos, fasta_seq_neg, fasta_fold_neg, cv, modele, mesure)
        # print("here : ", scores['test_score'])
        # print("Heeeere : ", mean(scores['test_score']))
        moyennes.append(mean(scores['test_score']))
        ecarts_types.append(stdev(scores['test_score']))
        
    return moyennes, ecarts_types


In [177]:
moyennes, ecarts_types = comparer_modeles(
                            fasta_seq_pos,
                            fasta_fold_pos,
                            fasta_seq_neg,
                            fasta_fold_neg,
                            cv=10,
                            modeles=["lsvc", "nsvc", "mnb", "gnb", "dtc"],
                            mesure="f1_weighted")
print('Moyennes : ',moyennes)
print('Ecarts types : ',ecarts_types)


Moyennes :  [0.9370394433691951, 0.9405545973843705, 0.8840511856564488, 0.9209123298171367, 0.9132475004074386]
Ecarts types :  [0.012329501648326564, 0.013867279240497342, 0.0033856050485683, 0.02241067508487675, 0.018469044664624915]


6.1 Comparer les modèles avec les précurseurs de l'humain

In [178]:
# Modèles à comparer :
comp_modeles = ["lsvc", "nsvc", "mnb", "gnb", "dtc"]

In [179]:
# Jeu de données 1
# ################
# séquences du set négatif générées par brassage aléatoire des nucléotides des séquences réelles

seq_pos = "Fasta/hsa_hairpin.fasta"
fold_pos = "Fasta/repliments/hsa_hairpin_fold.fasta"

seq_neg = "Fasta/hsa_hairpin_neg1.fasta"
fold_neg = "Fasta/repliments/hsa_hairpin_neg1_fold.fasta"

cv_iterations = 10
mesure_cv = "f1_weighted"

# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations, mesure=mesure_cv)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.937	0.012
nsvc	: 0.941	0.014
mnb	: 0.884	0.003
gnb	: 0.921	0.022
dtc	: 0.910	0.017


In [180]:
# Jeu de données 2
# ################
# séquences du set négatif générées par relocalisation de sous-séquences dans les séquences réelles

seq_pos = "Fasta/hsa_hairpin.fasta"
fold_pos = "Fasta/repliments/hsa_hairpin_fold.fasta"

seq_neg = "Fasta/hsa_hairpin_neg2.fasta"
fold_neg = "Fasta/repliments/hsa_hairpin_neg2_fold.fasta"

cv_iterations = 10
mesure_cv = "f1_weighted"

# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.884	0.003
nsvc	: 0.884	0.003
mnb	: 0.884	0.003
gnb	: 0.869	0.011
dtc	: 0.845	0.018


Question :
    Quelle méthode et quels modèles sont les plus efficaces pour discriminer entre les précurseurs réels et les pseudo-précurseurs ?

6.2 Comparer les modèles avec les précuseurs des animaux et des plantes

In [181]:
# Jeu de donnée 3
# ################

seq_pos = "Fasta/Animal/animal_hairpin.fasta"
fold_pos = "Fasta/Animal/repliments/animal_hairpin_fold.fasta"

seq_neg = "Fasta/Plant/plant_hairpin.fasta"
fold_neg = "Fasta/Plant/repliments/plant_hairpin_fold.fasta"

cv_iterations = 10
mesure_cv = "f1_weighted"


# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.887	0.050
nsvc	: 0.892	0.045
mnb	: 0.680	0.004
gnb	: 0.891	0.049
dtc	: 0.871	0.035


Question
Les modèles entrainés avec les triplets peuvent-ils faire la discrimination entre les précurseurs des animaux et ceux des plantes ?